In [ ]:
# import bs4
import requests
import numpy as np
import pandas as pd
import datetime as dt
from matplotlib import style
import matplotlib.pyplot as plt
import pandas_datareader.data as web
# from bs4 import BeautifulSoup

# We want to download the stock data somewhere first. In this example we are getting stock data from yahoo finance.

In [ ]:
style.use('ggplot')
start = dt.datetime(2000,1,1)
end = dt.datetime(2020,3,1)
df = web.DataReader('AAPL','yahoo', start, end)

In [ ]:
df

In [ ]:
style.use('ggplot')
start = dt.datetime(2000,1,1)
end = dt.datetime(2020,3,1)
df = web.DataReader('TSLA','yahoo', start, end)

# Now we want to look at Tesla's stock data, because trading Tesla is way more fun :)

In [ ]:
df

In [ ]:
df_stats = df.describe()
df_stats

# Data that we get from yahoo for free is not as good as the other data that you actually buy from data providers, so in this example we'll simply work on the closing price of the stock.

In [ ]:
daily_close = df['Close']
daily_close

In [ ]:
daily_close.shape

In [ ]:
daily_close[0]

In [ ]:
plt.figure(figsize=(10,8))
plt.xlabel('Time')
plt.ylabel('TSLA Price')
plt.title('TSLA daily close price', fontdict={'fontname': 'Comic Sans MS', 'fontsize': 20})
plt.plot(daily_close, 'r', label='TSLA close')

# From the graph and stock closing price series above, we know that if you hold TSLA stock from its IPO till now, you would have earn almost 30 times the money you invested as the price increased from 23.8999 to 667.98999. But what's a reason not to do so?

In [ ]:
high_watermark = 0
max_drawdown = 0
cur_maxdd = 0
high_watermark_dt = 0
dd_start = 0
dd_end = 0
for i in range(daily_close.size):
    if daily_close[i]>= high_watermark:
        high_watermark = daily_close[i]
        high_watermark_dt = df.index[i]
    cur_maxdd = (high_watermark -daily_close[i])/high_watermark
    if cur_maxdd > max_drawdown:
        max_drawdown = cur_maxdd
        dd_start = high_watermark_dt
        dd_end = df.index[i]
print('the max drawdown of tsla stock is:', max_drawdown)
print('the drawdown starts at', dd_start, ',ends at', dd_end)

In [ ]:
tsla_drawdown = web.DataReader('TSLA','yahoo', dd_start, dd_end)

In [ ]:
plt.figure(figsize=(10,8))
plt.xlabel('Time')
plt.ylabel('TSLA Price')
plt.title('TSLA drawdown period', fontdict={'fontname': 'Comic Sans MS', 'fontsize': 20})
plt.plot(tsla_drawdown['Close'], 'r', label='TSLA drawdown')

# A good way to protect your capital from this is to have a portfolio and actively adjust it, like a Hedgefund does, but let's suppose that you are a die-hard TSLA trader so it makes strategy development in this example a bit simpler :)

In [ ]:
df['SMA_50'] = df['Close'].rolling(50).mean()
df['SMA_200'] = df['Close'].rolling(200).mean()
previous_50 = df['SMA_50'].shift(1)
previous_200 = df['SMA_200'].shift(1)
df['SMA_diff'] = df['SMA_200'] - df['SMA_50']

crossing = (((df['SMA_50'] <= df['SMA_200']) & (previous_50 >= previous_200)) 
            | ((df['SMA_50'] >= df['SMA_200']) & (previous_50 <= previous_200)))

np.where(crossing == True)

In [ ]:
cashes = [0]
cash = 0
position = 0
for cross_point in np.where(crossing == True)[0]:    
    # if SMA_200 > SMA_50, buy. otherwise, sell.
    signal = np.sign(df['SMA_diff'].iloc[cross_point])
    
    cash += -1*(signal*df['Close'].iloc[cross_point])
    position += signal
    
    if position == 0:
        cashes.append(cash)
    
cashes = pd.Series(cashes)

plt.figure(figsize=(10,8))
plt.xlabel('Time')
plt.ylabel('your capital')
plt.title('your pnl', fontdict={'fontname': 'Comic Sans MS', 'fontsize': 20})
plt.plot(cashes, 'r', label='your pnl')

# Now we have a strategy that renders a much more smoother pnl curve(if we only look at how cash changes over the trades we make)!

In [ ]:
import matplotlib.image as mpimg 
import matplotlib.pyplot as plt 
plt.figure(figsize=(10,10))
img = mpimg.imread('stonks_man.png') 
plt.imshow(img) 

# But what if we change our way of backtesting a little bit?

In [ ]:
cashes = [0]
cash = 0
position = 0
df['SMA_50'] = df['Close'].rolling(50).mean()
df['SMA_200'] = df['Close'].rolling(200).mean()
df['SMA_diff'] = df['SMA_200'] - df['SMA_50']
# df.iloc[48]
# print(df)
for i in range(df['SMA_50'].size):
    if i > 199:
        # we start trading after we have data on both 200-day SMA and 50-day SMA
        # if SMA_200 > SMA_50, buy. otherwise, sell.
        if (df['SMA_50'][i] <= df['SMA_200'][i]) and (df['SMA_50'][i-1] >= df['SMA_200'][i-1]):
                cash += -df['Close'][i]
                position += 1
        if (df['SMA_50'][i] >= df['SMA_200'][i]) and (df['SMA_50'][i-1] <= df['SMA_200'][i-1]):
                cash += df['Close'][i]
                position += -1
        if position == 0:
            cashes.append(cash)
                                            
cashes = pd.Series(cashes)

plt.figure(figsize=(10,8))
plt.xlabel('Time')
plt.ylabel('your capital')
plt.title('your pnl', fontdict={'fontname': 'Comic Sans MS', 'fontsize': 20})
plt.plot(cashes, 'r', label='your pnl')

# That still looks nice, but what if we only look at our capital over time?

In [ ]:
capitals = [0]
capital = 0
position = 0
df['SMA_50'] = df['Close'].rolling(50).mean()
df['SMA_200'] = df['Close'].rolling(200).mean()
df['SMA_diff'] = df['SMA_200'] - df['SMA_50']
# df.iloc[48]
# print(df)
for i in range(df['SMA_50'].size):
    if i > 199:
        # we start trading after we have data on both 200-day SMA and 50-day SMA
        # if SMA_200 > SMA_50, buy. otherwise, sell.
        if (df['SMA_50'][i] <= df['SMA_200'][i]) and (df['SMA_50'][i-1] >= df['SMA_200'][i-1]):
                capital += -df['Close'][i]
                position += 1
        if (df['SMA_50'][i] >= df['SMA_200'][i]) and (df['SMA_50'][i-1] <= df['SMA_200'][i-1]):
                capital += df['Close'][i]
                position += -1
        capitals.append(capital)
                                            
capitals = pd.Series(capitals)

plt.figure(figsize=(10,8))
plt.xlabel('Time')
plt.ylabel('your capital')
plt.title('your pnl', fontdict={'fontname': 'Comic Sans MS', 'fontsize': 20})
plt.plot(capitals, 'r', label='your pnl')